In [7]:
#pip install pytube

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
from pytube import YouTube
import time

2023-11-15 20:58:44.476350: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

Make predictions - single pose

In [3]:
class Pose:
    def __init__(self, class_name):
        self.class_name = class_name
        self.pose_data = []

    def add_pose_data(self, csv_file, class_name, pose_landmarks):
        pose_row = [class_name]
        for landmark in pose_landmarks:
            x = landmark.x if landmark is not None else 0
            y = landmark.y if landmark is not None else 0
            z = landmark.z if landmark is not None else 0
            visibility = landmark.visibility if landmark is not None else 0
            pose_row.extend([x, y, z, visibility])
        pose_row.extend([x, y, z, visibility])
        csv_file.writerow(pose_row)

csv_file = open('pole_pose_data.csv', 'a', newline='')
csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

In [4]:
# Time
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

In [5]:
def process_and_record_coords(video_url, class_name, start_time_str, end_time_str,  output_folder):
    yt = YouTube(video_url)
    output_path = yt.streams.filter(adaptive=True, file_extension="mp4").first().download(output_path=output_folder)
    start_time_seconds = time_to_seconds(start_time_str)
    end_time_seconds = time_to_seconds(end_time_str)

    cap = cv2.VideoCapture(output_path)
    pose = Pose(class_name)
    key_held = False

    # Set the video capture position to the start time in seconds
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time_seconds * 1000)

    # Get the frame rate and frame count
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        pose = Pose(class_name)
        start_time = time.time()

        # Calculate the frame number at the end time
        end_frame_number = int(end_time_seconds * frame_rate)

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            current_time = time.time()
            elapsed_time = current_time - start_time

            # Check if the elapsed time is within the specified time range
            if elapsed_time >= 0:
                # Rest of your code for processing and recording pose data
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)

                # Recolor image back to BGR for rendering
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Draw landmarks for full body
                mp_drawing = mp.solutions.drawing_utils

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

                if results.pose_landmarks is not None:
                    pose.add_pose_data(csv_writer, class_name, results.pose_landmarks.landmark)

                cv2.imshow('Video Feed', image)

            # Control frame playback based on frame rate
            if cap.get(cv2.CAP_PROP_POS_FRAMES) >= end_frame_number:
                break

            # Check if the 'q' key is pressed to exit
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

    # Close the CSV file
    csv_file.close()

In [6]:
# # Load a YouTube video
# video_url = 'https://www.youtube.com/watch?v=VYyuzQxlYtc'  # Replace with your video URL
# start_time_str = '00:01:35'
# end_time_str = '00:01:43'
# class_name = 'cupid'

# # Download the video using pytube
# output_folder = 'YouTube Files'  # Save the downloaded video to a file

In [7]:
# make list of vars
with open('yt_pose_thing.csv', 'r') as csv_file:
    video_data = []
    for row in csv_file:
        row = row.rstrip().split(",")
        print(row)
        video_data.append(row)

['https://www.youtube.com/watch?v=L0OglVdoWP4', 'inside_leg_hang', '00:01:04', '00:01:14', 'YouTubeFiles']
['https://www.youtube.com/watch?v=4FJTxDb8t7k&t=2s', 'inside_leg_hang', '00:02:28', '00:02:44', 'YouTubeFiles']
['https://www.youtube.com/watch?v=75LetTNwfzc', 'inside_leg_hang', '00:01:39', '00:01:53', 'YouTubeFiles']
['https://www.youtube.com/watch?v=9A9mDWyhOgs', 'ayesha', '00:01:14', '00:01:16', 'YouTubeFiles']
['https://www.youtube.com/watch?v=GuMvp97eg7o', 'ayesha', '00:00:19', '00:00:21', 'YouTubeFiles']
['https://www.youtube.com/watch?v=wYe5M6Tq-WA', 'ayesha', '00:00:19', '00:00:22', 'YouTubeFiles']
['https://www.youtube.com/watch?v=kaFNVSbmzfQ', 'ayesha', '00:00:38', '00:00:41', 'YouTubeFiles']
['https://www.youtube.com/watch?v=OB7qtwC8GXk', 'cupid', '00:02:38', '00:03:11', 'YouTubeFiles']
['https://www.youtube.com/watch?v=VYyuzQxlYtc', 'cupid', '00:01:35', '00:01:43', 'YouTubeFiles']
['https://www.youtube.com/watch?v=pY-PQvCqFP8', 'cupid', '00:00:43', '00:00:47', 'YouTub

In [8]:
# for video_url, class_name, start_time_str, end_time_str, output_folder in list_of_targets:
#     yeet(video_url, start_time_str, end_time_str, class_name, output_folder)

In [43]:
process_and_record_coords(video_data[7][0], video_data[4][1], video_data[4][2], video_data[4][3],  video_data[4][4])

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv('pole_pose_data.csv')

In [23]:
df

,inside_leg_hang,0.4366028606891632,0.542475700378418,-0.10462462902069092,0.9999910593032837,0.4324156939983368,0.5513375997543335,-0.092565156519413,0.9999802112579346,0.43212997913360596,...,0.08264073729515076,0.9785600304603577,0.49131065607070923,0.26248598098754883,0.06380908191204071,0.8992204070091248,0.49131065607070923.1,0.26248598098754883.1,0.06380908191204071.1,0.8992204070091248.1
0,inside_leg_hang,0.436603,0.542838,-0.134355,0.999991,0.432425,0.552526,-0.122667,0.999980,0.432140,...,0.096959,0.977335,0.500443,0.266741,0.093907,0.889982,0.500443,0.266741,0.093907,0.889982
1,inside_leg_hang,0.436241,0.544218,-0.123524,0.999991,0.432247,0.554240,-0.111519,0.999981,0.431976,...,0.093235,0.977546,0.505931,0.267098,0.095618,0.891673,0.505931,0.267098,0.095618,0.891673
2,inside_leg_hang,0.436059,0.546924,-0.117539,0.999992,0.432155,0.557419,-0.104537,0.999981,0.431889,...,0.091441,0.977786,0.508739,0.266979,0.103328,0.894549,0.508739,0.266979,0.103328,0.894549
3,inside_leg_hang,0.435885,0.548757,-0.098494,0.999992,0.432046,0.559295,-0.085311,0.999982,0.431798,...,0.089510,0.978273,0.509666,0.266968,0.103421,0.898890,0.509666,0.266968,0.103421,0.898890
4,inside_leg_hang,0.435824,0.551793,-0.099775,0.999992,0.432001,0.562162,-0.086712,0.999981,0.431751,...,0.091831,0.978235,0.510495,0.266902,0.113410,0.901649,0.510495,0.266902,0.113410,0.901649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,cupid,0.696312,0.435068,-0.273823,0.999832,0.703039,0.431271,-0.275997,0.999788,0.706943,...,-0.173416,0.982986,0.442501,0.561784,0.283208,0.440920,0.442501,0.561784,0.283208,0.440920
1303,cupid,0.696193,0.434532,-0.268472,0.999799,0.702702,0.430818,-0.270325,0.999762,0.706456,...,-0.177263,0.982919,0.442623,0.561610,0.281803,0.443309,0.442623,0.561610,0.281803,0.443309
1304,cupid,0.695205,0.434348,-0.261434,0.999775,0.701574,0.430655,-0.263809,0.999746,0.705211,...,-0.176395,0.982660,0.444139,0.561307,0.284797,0.440591,0.444139,0.561307,0.284797,0.440591
1305,cupid,0.694057,0.434385,-0.275862,0.999757,0.700574,0.430677,-0.276529,0.999734,0.704226,...,-0.177656,0.982594,0.444587,0.561230,0.285879,0.441538,0.444587,0.561230,0.285879,0.441538


In [24]:
X = df.drop('inside_leg_hang', axis=1) # features
y = df['inside_leg_hang'] # target value

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [26]:
y_test

1106              cupid
1287              cupid
363     inside_leg_hang
504     inside_leg_hang
805               cupid
             ...       
541              ayesha
380     inside_leg_hang
224     inside_leg_hang
271     inside_leg_hang
535     inside_leg_hang
Name: inside_leg_hang, Length: 393, dtype: object

## 3.2 Train Machine Learning Classification Model

In [27]:
from sklearn.pipeline import make_pipeline #build ml pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [31]:
#seperate ML pipelines
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [32]:
list(pipelines.values())

[Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])]

In [33]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [34]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [35]:
fit_models['rf'].predict(X_test)

array(['cupid', 'cupid', 'inside_leg_hang', 'inside_leg_hang', 'cupid',
       'cupid', 'ayesha', 'cupid', 'inside_leg_hang', 'cupid', 'cupid',
       'cupid', 'cupid', 'cupid', 'cupid', 'cupid', 'cupid', 'cupid',
       'ayesha', 'cupid', 'ayesha', 'inside_leg_hang', 'cupid', 'cupid',
       'cupid', 'inside_leg_hang', 'cupid', 'inside_leg_hang',
       'inside_leg_hang', 'cupid', 'ayesha', 'cupid', 'cupid', 'cupid',
       'cupid', 'cupid', 'cupid', 'cupid', 'inside_leg_hang', 'cupid',
       'inside_leg_hang', 'cupid', 'inside_leg_hang', 'inside_leg_hang',
       'inside_leg_hang', 'inside_leg_hang', 'cupid', 'cupid',
       'inside_leg_hang', 'ayesha', 'cupid', 'cupid', 'cupid', 'cupid',
       'inside_leg_hang', 'cupid', 'cupid', 'ayesha', 'cupid', 'cupid',
       'inside_leg_hang', 'inside_leg_hang', 'inside_leg_hang',
       'inside_leg_hang', 'inside_leg_hang', 'inside_leg_hang', 'cupid',
       'inside_leg_hang', 'cupid', 'inside_leg_hang', 'cupid',
       'inside_leg_hang', '

In [37]:
## Check class distribution for imbalances

class_distribution = df['inside_leg_hang'].value_counts()
print("class distribution")
print(class_distribution)

class distribution
inside_leg_hang
cupid              627
inside_leg_hang    540
ayesha             140
Name: count, dtype: int64


# 3.3 Eval and serialize mode

In [38]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

In [39]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [40]:
fit_models['rf'].predict(X_test)

array(['cupid', 'cupid', 'inside_leg_hang', 'inside_leg_hang', 'cupid',
       'cupid', 'ayesha', 'cupid', 'inside_leg_hang', 'cupid', 'cupid',
       'cupid', 'cupid', 'cupid', 'cupid', 'cupid', 'cupid', 'cupid',
       'ayesha', 'cupid', 'ayesha', 'inside_leg_hang', 'cupid', 'cupid',
       'cupid', 'inside_leg_hang', 'cupid', 'inside_leg_hang',
       'inside_leg_hang', 'cupid', 'ayesha', 'cupid', 'cupid', 'cupid',
       'cupid', 'cupid', 'cupid', 'cupid', 'inside_leg_hang', 'cupid',
       'inside_leg_hang', 'cupid', 'inside_leg_hang', 'inside_leg_hang',
       'inside_leg_hang', 'inside_leg_hang', 'cupid', 'cupid',
       'inside_leg_hang', 'ayesha', 'cupid', 'cupid', 'cupid', 'cupid',
       'inside_leg_hang', 'cupid', 'cupid', 'ayesha', 'cupid', 'cupid',
       'inside_leg_hang', 'inside_leg_hang', 'inside_leg_hang',
       'inside_leg_hang', 'inside_leg_hang', 'inside_leg_hang', 'cupid',
       'inside_leg_hang', 'cupid', 'inside_leg_hang', 'cupid',
       'inside_leg_hang', '